In [1]:
from numcosmo_py import Ncm, Nc, GObject

import numpy as np
import math
import sys
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

from richness_mass_calib import create_richness_mass_calib

In [2]:
#Settings
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [3]:
# Data
DC2_halos_m200c = fits.open(
    "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits"
)
dt_halos = Table(DC2_halos_m200c[1].data)
# dt_halos['redshift_true', 'mass_fof', 'm200c', 'richness']

In [4]:
rmdata = create_richness_mass_calib(dt_halos)
ascaso = Nc.ClusterMassAscaso()
mset = Ncm.MSet()
mset.set(ascaso)

rmdata.m2lnL_val(mset)

13233701.307415523

In [5]:
dset = Ncm.Dataset.new()
dset.append_data(rmdata)

lh = Ncm.Likelihood.new(dset)
mset.param_set_all_ftype(Ncm.ParamType.FREE)

fit = Ncm.Fit.new(
    Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL
)

fit.run(Ncm.FitRunMsgs.SIMPLE)

#----------------------------------------------------------------------------------
# Model fitting. Interating using:
#  - solver:            NLOpt:ln-neldermead
#  - differentiation:   Numerical differentiantion (central)
#..............................................................................................................
#  Minimum found with precision: |df|/f =  1.00000e-08 and |dx| =  1.00000e-05
#  Elapsed time: 00 days, 00:00:18.5411220
#  iteration            [001014]
#  function evaluations [001016]
#  gradient evaluations [000000]
#  degrees of freedom   [454469]
#  m2lnL     =     391365.650978094 (     391365.65 )
#  Fit parameters:
#     4.08763247333783     0.538092645909241    0.931306629373459    0.674689968522022    0.0645030252979038  -0.244363116863723   


True

In [ ]:
fit.log_info()

In [ ]:
fit.numdiff_m2lnL_covar()  # covariance matrix

fit.log_covar()  # Prints to the log file the names and indices of the fitted parameters, their best-fit
# values, standard deviations and correlation matrix.

In [ ]:
gtkern = Ncm.MSetTransKernGauss.new(0)
mcmc = Ncm.FitMCMC.new(fit, gtkern, Ncm.FitRunMsgs.SIMPLE)

In [ ]:
cov = fit.fstate.covar.dup()
cov.scale(2.0)
gtkern.set_cov(cov)

In [ ]:
# mcmc.set_data_file("MCMC_ascaso_test.fits") #First fit
mcmc.set_data_file("MCMC_ascaso_test2.fits")  # For new fit

mcmc.start_run()
mcmc.run_lre(1000, 1.0e-3)
mcmc.end_run()
mcmc.mean_covar()

In [ ]:
fit.log_covar()

In [ ]:
# First Results (23-10-2023)
# ----------------------------------------------------------------------------------
# NcmMSet parameters covariance matrix
#                                                   -------------------------------------------------------------------------------------------
#    mup0[07000:00] =  4.422       +/-  0.002796    |  1           |  0.9095      | -0.08324     | -0.181       | -0.09781     | -0.03739     |
#    mup1[07000:01] =  0.6794      +/-  0.0009389   |  0.9095      |  1           |  0.2498      | -0.1644      | -0.1138      | -0.06086     |
#    mup2[07000:02] =  1.25        +/-  0.00286     | -0.08324     |  0.2498      |  1           | -0.01353     | -0.02935     | -0.02563     |
# sigmap0[07000:03] =  0.1588      +/-  0.0004555   | -0.181       | -0.1644      | -0.01353     |  1           | -0.4279      | -0.7289      |
# sigmap1[07000:04] = -0.1213      +/-  0.0002001   | -0.09781     | -0.1138      | -0.02935     | -0.4279      |  1           |  0.5469      |
# sigmap2[07000:05] = -0.1209      +/-  0.002002    | -0.03739     | -0.06086     | -0.02563     | -0.7289      |  0.5469      |  1           |
#                                                   -------------------------------------------------------------------------------------------